# Visualize extracted poles

In order to verify the results and spot potential issues, we can visualize individual objects. We take the output of `Extract_poles.ipynb` as input. The visualization renders the object from two directions (X and Y axis), along with a 3D projection.

In [ ]:
import os
import pandas as pd
import geopandas as gpd
import numpy as np
import laspy

import set_path  # add project src to path
from upc_analysis import visualization

import config as cf  # use config or config_azure

### Load poles data

In [ ]:
# Get extracted poles (after filtering)
poles_df = pd.read_csv(cf.output_file_filter)
poles_df.sort_values('tilecode', inplace=True)
print('# poles: ', len(poles_df))

### Loop over all poles

In [ ]:
# Define and create out folder for images
if not os.path.isdir(cf.img_out_folder):
    os.makedirs(cf.img_out_folder + 'object_all_axes/')
    os.makedirs(cf.img_out_folder + 'object_per_axis/x/')
    os.makedirs(cf.img_out_folder + 'object_per_axis/y/')

In [ ]:
# Create 3 images for each of the extracted poles
visualization.create_images_for_poles(poles_df, cf.dataset_folder, cf.pred_folder,
                                      cf.img_out_folder, cf.prefix, cf.prefix_pred)

### Check locations

In [ ]:
gdf = gpd.GeoDataFrame(poles_df, geometry=gpd.points_from_xy(poles_df['rd_x'], poles_df['rd_y']))
gdf.plot()